### 📌 Visão Geral do Projeto  

O **Gradio** é uma biblioteca Python que facilita a criação de interfaces de usuário para modelos de Machine Learning, processamento de texto, imagens e muito mais. Com poucas linhas de código, podemos desenvolver uma UI interativa e compartilhá-la via link sem necessidade de um servidor complexo.  

Neste projeto, vamos construir uma aplicação simples que usa **Gradio** para criar uma interface interativa. A aplicação permitirá que o usuário envie um texto e receba uma resposta processada. Podemos integrar isso a um modelo de NLP, como um chatbot usando **OpenAI GPT** ou até mesmo aplicar transformações básicas ao texto, como converter para maiúsculas.  

---

### 🔹 **Objetivo do Projeto**  

Criar uma interface interativa usando **Gradio** para processar entrada de texto. O usuário insere um texto e recebe uma resposta processada, podendo ser um simples transformador de texto ou um chatbot.  

**O que aprenderemos?**  
✅ Como instalar e configurar o Gradio.  
✅ Como criar uma interface básica com entrada e saída de texto.  
✅ Como integrar Gradio com uma função de processamento.  
✅ Como compartilhar a aplicação na web facilmente.  

# **Importando as bibliotecas**

In [2]:
# Biblioteca para interagir com o sistema operacional e acessar variáveis de ambiente
import os

# Biblioteca para enviar requisições HTTP (como GET e POST) e obter dados da web
import requests

# BeautifulSoup é usada para fazer o parsing e extração de dados de HTML (ideal para scraping)
from bs4 import BeautifulSoup

# Tipagem para listas, usada para garantir que variáveis sejam listas de tipos específicos
from typing import List

# Biblioteca para carregar variáveis de ambiente a partir de arquivos .env
from dotenv import load_dotenv

# Biblioteca para interação com a API da OpenAI, incluindo geração de textos com GPT
from openai import OpenAI

# Gradio é usado para criar interfaces de usuário interativas de forma simples e rápida
import gradio as gr


In [3]:
# Carrega as variáveis de ambiente a partir do arquivo .env
# O parâmetro 'override=True' permite sobrescrever variáveis já carregadas
load_dotenv(override=True)

# Obtém a chave de API do OpenAI da variável de ambiente
openai_api_key = os.getenv('OPENAI_API_KEY')

# Verifica se a chave de API foi carregada corretamente
if openai_api_key:
    # Se a chave de API existir, imprime os primeiros 8 caracteres da chave
    print(f"A chave de API OpenAI existe e começa {openai_api_key[:8]}")
else:
    # Caso contrário, informa que a chave de API não foi definida
    print("Chave de API OpenAI não definida")


A chave de API OpenAI existe e começa sk-proj-


In [4]:
# Conectando a API da OpenAI, 

openai = OpenAI()

In [5]:
system_message = "Você é um assistente útil"

In [6]:
# Função que interage com a API do GPT da OpenAI para gerar respostas baseadas em um prompt de usuário
def message_gpt(prompt):
    # A lista 'messages' contém as mensagens de contexto e do usuário para a interação com o modelo GPT
    messages = [
        {"role": "system", "content": system_message},  # Mensagem do sistema, define o comportamento do modelo
        {"role": "user", "content": prompt}  # Mensagem do usuário, o prompt que será respondido pelo modelo
    ]
    
    # Envia a requisição para a API da OpenAI, pedindo uma completão com o modelo especificado e as mensagens fornecidas
    completion = openai.chat.completions.create(
        model='gpt-4o-mini',  # Modelo GPT-4 que será utilizado para gerar a resposta
        messages=messages,  # Lista de mensagens fornecidas para o modelo
    )
    
    # Retorna o conteúdo da resposta gerada pelo modelo GPT, extraindo o texto da primeira opção retornada
    return completion.choices[0].message.content


# Hora da interface do usuário

In [7]:
# Função simples que converte o texto para letras maiúsculas
def shout(text):
    # Exibe uma mensagem no console mostrando o texto que foi passado para a função
    print(f"Shout foi chamado com a entrada {text}") 
    
    # Retorna o texto convertido para letras maiúsculas
    return text.upper()


In [8]:
# A simplicidade do gradio. Isso pode aparecer no "modo claro" 
gr.Interface(fn=shout, inputs="textbox", outputs="textbox").launch()

* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


In [9]:
# Adicionar share=True significa que ele pode ser acessado publicamente
# Uma hospedagem mais permanente está disponível usando uma plataforma chamada Spaces from HuggingFace
# NOTA: Alguns softwares antivírus e firewalls corporativos podem não gostar de você usar share=True. 
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(share=True)

* Running on local URL:  http://127.0.0.1:7864
* Running on public URL: https://b5a70640ba34293fab.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [10]:
# Adicionar inbrowser=True abre uma nova janela do navegador automaticamente

gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


## Forçando o modo escuro

Gradio aparece no modo claro ou escuro dependendo das configurações do navegador e do computador. Existe uma maneira de forçar o gradio a aparecer no modo escuro, mas o Gradio não recomenda isso, pois deve ser uma preferência do usuário (principalmente por motivos de acessibilidade). Mas se você deseja forçar o modo escuro em suas telas, veja abaixo como fazer isso.


In [11]:
# Defina esta variável e então passe js=force_dark_mode ao criar a Interface

force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never", js=force_dark_mode).launch()

* Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.


# Integrando o Gradio ao GPT

In [13]:
# Função que interage com o modelo GPT de forma contínua (streaming)
def stream_gpt(prompt):
    # Define a lista de mensagens, incluindo a mensagem do sistema e a entrada do usuário
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
    ]

    # Envia a solicitação para o modelo GPT-4o-mini, ativando o modo de streaming
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True  # Habilita o streaming de respostas
    )

    result = ""  # Inicializa a variável para armazenar a resposta gerada

    # Itera sobre os blocos de resposta recebidos do modelo
    for chunk in stream:
        # Adiciona cada novo trecho da resposta ao resultado final
        result += chunk.choices[0].delta.content or ""

        # Utiliza 'yield' para transmitir os dados progressivamente
        yield result


In [15]:
# Criando a interface do usuário com Gradio
view = gr.Interface(
    fn=stream_gpt,  # Define a função que processa a entrada do usuário e retorna a resposta do modelo GPT
    inputs=[gr.Textbox(label="Your message:")],  # Campo de entrada de texto para o usuário digitar sua mensagem
    outputs=[gr.Markdown(label="Response:")],  # Saída formatada como Markdown para exibir a resposta do GPT
    flagging_mode="never"  # Desativa a opção de reportar respostas como inapropriadas
)

# Inicializa a interface web e a disponibiliza para uso
view.launch()


* Running on local URL:  http://127.0.0.1:7869

To create a public link, set `share=True` in `launch()`.
